In [1]:
import psycopg2

import pandas as pd
import numpy as np
import mariadb
# import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc
from datetime import datetime
from dateutil import parser

In [2]:
hydroponics_df_time_filtered = pd.read_csv('input/hydroponics_samples_.csv')

In [3]:
hydroponics_df_time_filtered

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,513306,513306,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,molybdenum,0.010,ppm
1,513305,513305,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,copper,0.014,ppm
2,513294,513294,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,calcium,5.430,ppm
3,513295,513295,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,magnesium,1.480,ppm
4,513304,513304,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,boron,0.026,ppm
...,...,...,...,...,...,...,...,...
685941,540124,540124,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,ph,6.040,NaN
685942,540126,540126,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,ammonium,34.000,ppm
685943,550140,550140,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,manganese,0.260,ppm
685944,550145,550145,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,sodium,28.500,ppm


In [4]:
hydroponics_df_time_filtered['analysis_name'] = [ str(i).split(",") for i in hydroponics_df_time_filtered['analysis_name'] ]

In [5]:
hydroponics_df_time_filtered= hydroponics_df_time_filtered.explode("analysis_name")

In [6]:
hydroponics_df_time_filtered

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,513306,513306,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,molybdenum,0.010,ppm
1,513305,513305,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,copper,0.014,ppm
2,513294,513294,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,calcium,5.430,ppm
3,513295,513295,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,magnesium,1.480,ppm
4,513304,513304,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,boron,0.026,ppm
...,...,...,...,...,...,...,...,...
685941,540124,540124,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,ph,6.040,NaN
685942,540126,540126,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,ammonium,34.000,ppm
685943,550140,550140,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,manganese,0.260,ppm
685944,550145,550145,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,sodium,28.500,ppm


In [7]:
hydroponics_df_time_filtered.analysis_name.value_counts()

analysis_name
Drain Water Analysis                                330034
Drip Water Analysis                                 267171
Substrate Analysis (Pumice) 1:1.5 Volume Extract     41032
Substrate Analysis (Cocos) 1:1.5 Volume Extract      24291
Recycle Water Analysis                               12618
Raw irrigation water analysis (for fertigation)      10181
Fluoride Analysis                                     1790
Heavy metals in Hydroponics                           1500
Fluoride Analysis (Media)                              713
Fluoride Analysis (Drain)                              695
Aluminium Analysis (Media)                             680
Fluoride Analysis (Drip)                               600
E coli & Total Coliforms                               213
Aqua regia - Selected elements (ICP) Analysis          158
Biochemical Oxygen Demand                              116
Chemical Oxygen Demand                                 116
Selected elements (ICP) Analysis          

In [8]:
df_ = hydroponics_df_time_filtered[['analysis_name','chemical_name','unit_name']].value_counts()
df_.to_csv("output/hydroponics_unit_per_chemical.csv")

In [9]:
df_ = df_.reset_index()

In [10]:
analysis_chem_unit_df = pd.DataFrame()
for analysis in df_['analysis_name'].unique():
    print(analysis)
    chems_count_ = df_.loc[df_['analysis_name'] == analysis]
    duplicates_chem = (chems_count_[chems_count_.duplicated(subset='chemical_name')])
    temp_analysis_chem_unit_df = pd.DataFrame()
    for index,row in duplicates_chem.iterrows():
        chemical = row['chemical_name']
        chems_count_2 = chems_count_.loc[chems_count_['chemical_name'] == chemical ]
        unit = chems_count_2.loc[chems_count_2['count'] == max(chems_count_2['count'])]['unit_name'].values[0]
        _ = pd.DataFrame({"analysis_name":[analysis],"chemical_name":[chemical], "unit_name":[unit]})
        temp_analysis_chem_unit_df = pd.concat([temp_analysis_chem_unit_df, _])
    analysis_chem_unit_df = pd.concat([analysis_chem_unit_df, temp_analysis_chem_unit_df])
analysis_chem_unit_df.to_csv("output/sgf_unit_per_chemical_decision.csv")

Drain Water Analysis
Drip Water Analysis
Substrate Analysis (Pumice) 1:1.5 Volume Extract
Substrate Analysis (Cocos) 1:1.5 Volume Extract
Recycle Water Analysis
Raw irrigation water analysis (for fertigation)
Fluoride Analysis
Heavy metals in Hydroponics
Fluoride Analysis (Drain)
Fluoride Analysis (Media)
Aluminium Analysis (Media)
Fluoride Analysis (Drip)
E coli & Total Coliforms
Aqua regia - Selected elements (ICP) Analysis
EC Analysis 
pH Analysis (Media)
Sodium Analysis
Biochemical Oxygen Demand
Chemical Oxygen Demand
pH Analysis
Biuret test
Selected elements (ICP) Analysis
Sample digestion
Alkalinity 
Turbidity
Sodium Analysis 
Hardness
Fluoride Analysis (cocos)
Total Dissolved Solids
Aluminium Analysis 
Aluminium Analysis
Fibre Analysis
Total Bacteria Count
pH
 EC & Sodium Analysis 
Bacterial test (9) in hydroponics
Organic Matter


In [11]:
df = pd.DataFrame()
for index, row in analysis_chem_unit_df.iterrows():
    analysis = row['analysis_name']
    chemical = row['chemical_name']
    unit = row['unit_name']
    temp_df = hydroponics_df_time_filtered.loc[(hydroponics_df_time_filtered['analysis_name'] == analysis) & (hydroponics_df_time_filtered['chemical_name'] == chemical) & (hydroponics_df_time_filtered['unit_name'] == unit)]
    df = pd.concat([df, temp_df])

In [12]:
df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
29,525705,525705,CC003HY0107,2023-03-29 10:46:00.000,Drain Water Analysis,bicarbonate,260.0,ppm
117,520391,520391,CO006HY0294,2023-03-30 15:13:00.000,Drain Water Analysis,bicarbonate,79.8,ppm
118,521381,521381,CO006HY0292,2023-03-30 15:13:00.000,Drain Water Analysis,bicarbonate,44.9,ppm
165,519730,519730,CE361HY0019,2023-03-30 15:59:00.000,Drain Water Analysis,bicarbonate,153.0,ppm
196,525725,525725,CE361HY0017,2023-03-30 15:59:00.000,Drain Water Analysis,bicarbonate,83.7,ppm
...,...,...,...,...,...,...,...,...
142541,648932,648932,CO148HY0139,2024-10-14 17:39:00.000,Bacterial test (9) in hydroponics,total_coliforms,100.0,cfu/100 ml
134070,653827,653827,CC370HY0004,2024-09-10 12:31:00.000,Bacterial test (9) in hydroponics,listeria_monocytogenes,0.0,cfu/ml
134070,653827,653827,CC370HY0004,2024-09-10 12:31:00.000,Bacterial test (9) in hydroponics,listeria_monocytogenes,0.0,cfu/ml
134060,653824,653824,CC370HY0004,2024-09-10 12:31:00.000,Bacterial test (9) in hydroponics,e_coli,0.0,cfu/ml


In [13]:
df.to_csv("output/hydroponics_analysis_cleaned.csv",chunksize=1000)